# Pengolahan Isyarat ECG untuk mendeteksi Aritmia menggunakan CNN+LSTM

# 1) Mendefinisikan Dataset

In [ ]:
!pip install PyWavelets

In [ ]:
# Masukkan library yang akan digunakan
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import csv
import itertools
import collections
import pywt
from scipy import stats
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Conv1D, AvgPool1D, Flatten, Dense, Dropout, Softmax
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.utils import plot_model
from keras import regularizers

%matplotlib inline

In [ ]:
# Load dataset
!pip install opendatasets
import opendatasets as od
od.download('https://www.kaggle.com/datasets/taejoongyoon/mitbit-arrhythmia-database')
path = '/content/mitbit-arrhythmia-database/mitbih_database/mitbih_database/'

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

In [ ]:
# Baca File
filenames = next(os.walk(path))[2]

# Bagi dan simpan file ke dalam format .csv , .txt
dataset = list()
annotations = list()
filenames.sort()

In [ ]:
# Memisahkan Data dan Anotasi
for f in filenames:
    filename, file_extension = os.path.splitext(f)

    # *.csv
    if(file_extension == '.csv'):
        dataset.append(path + filename + file_extension)

    # *.txt
    else:
        annotations.append(path + filename + file_extension)

In [ ]:
# Definisikan Path yang digunakan
dataset = [
    '/content/mitbit-arrhythmia-database/mitbih_database/100.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/101.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/102.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/103.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/104.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/105.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/106.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/107.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/108.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/109.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/111.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/112.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/113.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/114.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/115.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/116.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/117.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/118.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/119.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/121.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/122.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/123.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/124.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/200.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/201.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/202.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/203.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/205.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/207.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/208.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/209.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/210.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/212.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/213.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/214.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/215.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/217.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/219.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/220.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/221.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/222.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/223.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/228.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/230.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/231.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/232.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/233.csv',
    '/content/mitbit-arrhythmia-database/mitbih_database/234.csv',
]

annotations = [
    '/content/mitbit-arrhythmia-database/mitbih_database/100annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/101annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/102annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/103annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/104annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/105annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/106annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/107annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/108annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/109annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/111annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/112annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/113annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/114annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/115annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/116annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/117annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/118annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/119annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/121annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/122annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/123annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/124annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/200annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/201annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/202annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/203annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/205annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/207annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/208annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/209annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/210annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/212annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/213annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/214annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/215annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/217annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/219annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/220annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/221annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/222annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/223annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/228annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/230annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/231annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/232annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/233annotations.txt',
    '/content/mitbit-arrhythmia-database/mitbih_database/234annotations.txt'
]


# 2) Ekstraksi data dan Pra-Pemrosesan

In [ ]:
# Definisikan kelas dan data size yang digunakan
window_size = 180
maximum_counting = 10000

classes = ['N', 'L', 'R', 'A', 'V']
# Normal beat (N)
# Left bundle branch block beat (L)
# Right bundle branch block beat (R)
# Atrial premature beat (A)
# Premature ventricular contraction (V)
n_classes = len(classes)
count_classes = [0]*n_classes

X = list()
y = list()

In [ ]:
# Menghilangkan noise pada data
def denoise(data):
    w = pywt.Wavelet('sym4')
    maxlev = pywt.dwt_max_level(len(data), w.dec_len)
    threshold = 0.05 # Threshold untuk filter

    coeffs = pywt.wavedec(data, 'sym4', level=maxlev)
    for i in range(1, len(coeffs)):
        coeffs[i] = pywt.threshold(coeffs[i], threshold*max(coeffs[i]))

    datarec = pywt.waverec(coeffs, 'sym4')

    return datarec

In [ ]:
# Menampilkan Dataset
for r in range(len(dataset)):
    signals = []
    file_path = dataset[r]

    # Cek apakah file ada atau tidak
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        continue

    # Membaca sinyal dari dataset
    with open(file_path, 'rt') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row_index, row in enumerate(spamreader):
            if row_index > 0:
                signals.append(int(row[1]))

    # Atur plot grafik
    plt.rcParams["figure.figsize"] = (30,6)
    plt.rcParams['lines.linewidth'] = 1
    plt.rcParams['lines.color'] = 'r'
    plt.rcParams['axes.grid'] = True

    # Plot contoh sinyal mentah
    if r == 1:
        plt.title(dataset[1] + " Sinyal Mentah")
        plt.plot(signals[0:700])
        plt.show()

    # Plot contoh sinyal setelah denoising
    signals = denoise(signals)
    if r == 1:
        plt.title(dataset[1] + " Sinyal Setelah Denoising")
        plt.plot(signals[0:700])
        plt.show()

    # Plot an example of the signals
    signals = stats.zscore(signals)
    if r == 1:
        plt.title(dataset[1] + " Sinyal Ternormalisasi")
        plt.plot(signals[0:700])
        plt.show()

    # Membaca anotasi: Posisi R dan kelas aritmia
    example_beat_printed = False
    with open(annotations[r], 'r') as fileID:
        data = fileID.readlines()
        beat = []

        for d in range(1, len(data)):
            splitted = data[d].split()
            pos = int(splitted[1])
            arrhythmia_type = splitted[2]
            if arrhythmia_type in classes:
                arrhythmia_index = classes.index(arrhythmia_type)
                count_classes[arrhythmia_index] += 1
                if window_size <= pos < (len(signals) - window_size):
                    beat = signals[pos-window_size:pos+window_size]
                    if maximum_counting > count_classes[arrhythmia_index]:
                    # Plot an example of a beat
                      if r == 1 and not example_beat_printed:
                          plt.title("A Beat from " + dataset[1] + " Wave")
                          plt.plot(beat)
                          plt.show()
                          example_beat_printed = True

                      X.append(beat)
                      y.append(arrhythmia_index)

# Ukuran dataset
print(np.shape(X), np.shape(y))

In [ ]:
# Reshape
for i in range(0,len(X)):
        X[i] = np.append(X[i], y[i])

print(np.shape(X))

# 3) Distribusi Kelas

In [ ]:
# Distribusi kelas data latih
X_train_df = pd.DataFrame(X)
per_class = X_train_df[X_train_df.shape[1]-1].value_counts()
print(per_class)
plt.figure(figsize=(20,10))
my_circle=plt.Circle( (0,0), 0.7, color='white')
plt.pie(per_class, labels=['N', 'L', 'R', 'A', 'V'], colors=['tab:blue','tab:orange','tab:purple','tab:olive','tab:green'],autopct='%1.1f%%')
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.show()

# 4) Menyeimbangkan Kelas yang Digunakan

In [ ]:
# Memisahkan data berdasarkan kelas
df_L=X_train_df[X_train_df[X_train_df.shape[1]-1]==1]
df_R=X_train_df[X_train_df[X_train_df.shape[1]-1]==2]
df_A=X_train_df[X_train_df[X_train_df.shape[1]-1]==3]
df_V=X_train_df[X_train_df[X_train_df.shape[1]-1]==4]

# Ambil sampel dari kelas frekuensi tinggi untuk menyeimbangkan dengan kelas lainnya
df_N=(X_train_df[X_train_df[X_train_df.shape[1]-1]==0]).sample(n=5000,random_state=42)

# Upsampling kelas dengan frekuensi rendah
df_L_upsample=resample(df_L,replace=True,n_samples=5000,random_state=122)
df_R_upsample=resample(df_R,replace=True,n_samples=5000,random_state=123)
df_A_upsample=resample(df_A,replace=True,n_samples=5000,random_state=124)
df_V_upsample=resample(df_V,replace=True,n_samples=5000,random_state=125)

# Gabungkan data
X_train_df=pd.concat([df_N,df_L_upsample,df_R_upsample,df_A_upsample,df_V_upsample])


In [ ]:
# Distribusikan ulang data dan visualisasikan
per_class = X_train_df[X_train_df.shape[1]-1].value_counts()
print(per_class)
plt.figure(figsize=(10,5))
my_circle=plt.Circle( (0,0), 0.7, color='white')
plt.pie(per_class, labels=['N', 'L', 'R', 'A', 'V'], colors=['tab:blue','tab:orange','tab:purple','tab:olive','tab:green'],autopct='%1.1f%%')
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.show()

# 5) Membagi Data ke Training dan Testing

In [ ]:
# Membagi data ke dalam data latih dan data uji
train, test = train_test_split(X_train_df, test_size=0.25)
print("X_train : ", np.shape(train))
print("X_test  : ", np.shape(test))

In [ ]:
# Pisahkan label dari fitur
target_train=train[train.shape[1]-1]
target_test=test[test.shape[1]-1]

# Ubah target ke format kategori
train_y=to_categorical(target_train)
test_y=to_categorical(target_test)
print(np.shape(train_y), np.shape(test_y))

In [ ]:
# Pisahkan fitur dari dataset
train_x = train.iloc[:,:train.shape[1]-1].values
test_x = test.iloc[:,:test.shape[1]-1].values

# Reshape menyesuaikan model CNN + LSTM
train_x = train_x.reshape(len(train_x), train_x.shape[1],1)
test_x = test_x.reshape(len(test_x), test_x.shape[1],1)
print(np.shape(train_x), np.shape(test_x))

# 6) Membangun Arsitektur Model

## a) CNN untuk Ekstraksi Fitur

In [ ]:
from keras.layers import LSTM
# Definisikan Model
model = Sequential()

In [ ]:
# Convolutional layers for feature extraction
model.add(Conv1D(filters=16, kernel_size=13, padding='same', activation='relu', input_shape=(360, 1)))
model.add(AvgPool1D(pool_size=3, strides=2))
model.add(Conv1D(filters=32, kernel_size=15, padding='same', activation='relu'))
model.add(AvgPool1D(pool_size=3, strides=2))
model.add(Conv1D(filters=64, kernel_size=19, padding='same', activation='relu'))
model.add(AvgPool1D(pool_size=3, strides=2))
model.add(Conv1D(filters=128, kernel_size=19, padding='same', activation='relu'))
model.add(AvgPool1D(pool_size=3, strides=2))

## b) LSTM untuk Klasifikasi

In [ ]:
# LSTM layer for sequence processing
model.add(LSTM(units=128, return_sequences=False))

In [ ]:
model.add(Flatten())
model.add(Dropout(0.5))

# Dense layers for classification
model.add(Dense(35, kernel_regularizer=regularizers.l2(0.0001), bias_regularizer=regularizers.l2(0.0001)))
model.add(Dense(5, kernel_regularizer=regularizers.l2(0.0001), bias_regularizer=regularizers.l2(0.0001)))
model.add(Softmax())

In [ ]:
# Cetak Model
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# 7) Pelatihan Model

In [ ]:
# Bagi data menjadi training dan testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Cetak ukuran data uji
print(train_x.shape)

In [ ]:
# Latih model dengan epoch 10
history = model.fit(train_x, train_y, batch_size=36, epochs=10, verbose=1, validation_data=(test_x, test_y))

# 8) Evaluasi Kinerja

In [ ]:
# Tampilkan hasil evaluasi berupa akurasi dan loss
score = model.evaluate(test_x, test_y)

print('Testing data Loss:', score[0])
print('Testing data accuracy: {:.2f}%'.format(score[1]*100))

In [ ]:
# Hasil prediksi set pengujian dengan model terakhir yang dilatih
final_predictions = model.predict(test_x)
final_predictions = np.round(final_predictions).astype(int)

In [ ]:
# Hitung dan tampilkan hasil metric precision, recall, F1 score
from sklearn.metrics import classification_report
import numpy as np

y_pred = model.predict(test_x)
y_pred_classes = np.argmax(y_pred, axis=1)

if len(test_y.shape) > 1 and test_y.shape[1] > 1:
    test_y_classes = np.argmax(test_y, axis=1)
else:
    test_y_classes = test_y

print(classification_report(test_y_classes, y_pred_classes, target_names=classes))

In [ ]:
from sklearn.metrics import confusion_matrix
import plotly.express as px

# Hitung confusion matrix
cm = confusion_matrix(test_y_classes, y_pred_classes,
                      normalize='true'
                     )

labels = ['N', 'L', 'R', 'A', 'V']

cm_df = pd.DataFrame(cm, index=labels, columns=labels)

# Plot confusion matrix
cm_fig = px.imshow(cm_df,
                labels=dict(x="Predicted", y="True", color="Count"),
                x=labels,
                y=labels,
                text_auto=True,
                title='Confusion Matrix',
                color_continuous_scale='Blues'
               )
cm_fig.update_layout(title_x=0.5, width=800, height=600)
cm_fig.show()

# 9) Simpan Bobot Hasil Training

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Simpan Model
model.save('/content/drive/MyDrive/TPIB 2024/Model_CNN_LSTM.pth')

# Simpan Bobot Model (weights)
model.save_weights('/content/drive/MyDrive/TPIB 2024/Model_CNN_LSTM_weights.pth')